## Analiza zarobków analityków danych i zawodów pokrewnych na świecie z podziałem na stanowiska. 

#### Spis treści:
- []
- [Wnioski](#wnioski)

Górski Szymon, Szura Michał, Stano Paweł

TODO:
- read csv do dataframe
- clean data (sprawdzic w wykladzie) na przyklad dropnan/inan (opcjonalnie usunięcie wartości skrajnych)
- analiza: group by (np country/stanowisko/firma) + miary tendencji centralnej (oda, mediana, średnia)
- Wykres do powyzszego.

jak zdążymy:
- analiza wewnątrze dla wybranego przez usera kraj (np breakdown by stanowisko/firma)
- analiza per firma niezależnie od kraju
- analiza per stanowisko
- zaczynamy od pisania tego w funkcjach, jak zdążymy wrzucamy fnkcje na metody wewnętrzne obietu

In [46]:
import requests as re
import json
import base64
import zipfile
from io import BytesIO, StringIO

In [166]:
import pandas as pd
import csv
import unicodedata
salary_file = r'./salaries.csv'
dataset = []
with open(salary_file, newline='', encoding ='utf-8') as csvfile:
    spamreader = csv.reader(csvfile,delimiter='\t', quotechar=u'\xa0')
    for row in spamreader:
        row = [unicodedata.normalize('NFC',i)  for i in row]
        dataset.append(row)

salaries_df = pd.DataFrame(columns = dataset[0][:-1], data = dataset[1:])
salaries_df.head(25)

,role,country,company,company_rating,job_title,median,low,high,unit
0,Data Scientist,Japan,CTW,2.9,Data Scientist,"¥1,500,000",¥1M,¥18M,/yr
1,Data Scientist,Japan,FUJIFILM,3.7,Data Scientist,"¥5,000,000",¥5M,¥7M,/yr
2,Data Scientist,Japan,GMO Internet,3.3,Data Scientist,"¥6,500,000",¥7M,-,/yr
3,Data Scientist,Japan,Data4Cʼs,3,Data Scientist,"¥350,000",¥300K,¥402K,/mo
4,Data Scientist,Japan,Deloitte,4,Data Scientist,"¥6,250,000",¥5M,¥8M,/yr
5,Data Scientist,Japan,Indeed,4.1,Data Scientist,"¥20,500,000",¥20M,¥21M,/yr
6,Data Scientist,Japan,Microsoft,4.3,Data Scientist,"¥650,000",¥300K,¥1M,/mo
7,Data Scientist,Japan,EY,3.8,Data Scientist,"¥708,333",¥417K,¥1M,/mo
8,Data Scientist,Japan,LINE,3.9,Data Scientist,"¥9,750,000",¥8M,¥12M,/yr
9,Data Scientist,Japan,Fujitsu,3.7,Data Scientist,"¥8,750,000",¥8M,¥10M,/yr


In [175]:
class A:
  base_url = "https://www.kaggle.com/api/v1"

  def __init__(self, username: str, key: str, currency_symbols: str) -> None:
    self.username = username
    self.key = key
    self.currency_symbols = currency_symbols

    credentials = base64.b64encode(bytes(f"{self.username}:{self.key}", "ISO-8859-1")).decode("ascii")
    self.headers = {
      "Authorization": f"Basic {credentials}"
    }
    self.dataset = pd.DataFrame()

  def load_dataset(self, owner: str, dataset: str, filename: str, version: int, map: dict) -> None:
    url = f'{self.base_url}/datasets/download/{owner}/{dataset}?datasetVersionNumber={version}'
    res: re.Response 
    try:
      res = re.get(url, headers=self.headers, stream=True)
    except Exception as e:
      print(e)
      return 
    
    try:
      zf = zipfile.ZipFile(BytesIO(res.content))
      with zf.open(filename) as f:
        content = f.read().decode('utf-8')
        f = StringIO(content)
        set = []
        spamreader = csv.reader(f, delimiter='\t', quotechar=u'\xa0')
        for row in spamreader:
            row = [unicodedata.normalize('NFC',i)  for i in row]
            set.append(row)

        self.dataset = pd.DataFrame(columns = set[0][:-1], data = set[1:])
        self.dataset = self.dataset[~self.dataset.isin(['NA', '-']).any(axis=1)]
        self.dataset = self.dataset.dropna()
        for key, value in map.items():
          self.dataset[key] = self.dataset[key].astype(value)
    except Exception as e:
        print(e)
        return
  
  def process_data(self) -> None:
    # TODO: refactor (adjust to pandas 3.0)
    def convert_to_proper_number(value):
      multiplier = 1
      if 'K' in value:
          multiplier = 1_000
      elif 'M' in value:
          multiplier = 1_000_000
      value = value.replace('K', '')
      value = value.replace('M', '')
      
      return float(value) * multiplier
  
    currency_codes = self.dataset['median'].str.extract(r'^(\w+)', expand=False)
      
    self.dataset['Currency'] = currency_codes.map(self.currency_symbols)
    self.dataset['Currency'].fillna(currency_codes, inplace=True)
  
    symbols = self.dataset.loc[self.dataset['Currency'].isna(), 'median'].str.extract(r'([^\d\.\-KM])', expand=False).fillna('')
    self.dataset.loc[self.dataset['Currency'].isna(), 'Currency'] = symbols.map(self.currency_symbols)
        
    self.dataset['Currency'].fillna(symbols, inplace=True)
    
    self.dataset['median'] = self.dataset['median'].str.replace(r'[^\d\.\-KM]', '', regex=True).apply(convert_to_proper_number)
    self.dataset['low'] = self.dataset['low'].str.replace(r'[^\d\.\-KM]', '', regex=True).apply(convert_to_proper_number)
    self.dataset['high'] = self.dataset['high'].str.replace(r'[^\d\.\-KM]', '', regex=True).apply(convert_to_proper_number)

  def get_dataset(self) -> pd.DataFrame:
    return self.dataset
  
  def select_by_hour_rate(self) -> pd.DataFrame:
    def convert_to_hr_rate(value, unit):
      conversion_factors = {
        '/yr': 12 * 160,
        '/mo': 160,
        '/hr': 1
      }

      return value / conversion_factors[unit]
    
    for col in ['median', 'low', 'high']:
      self.dataset[col] = self.dataset.apply(lambda row: convert_to_hr_rate(row[col], row['unit']), axis=1)
    self.dataset['unit'] = '/hr'

    return self.get_dataset()
  
  def calculate_mean(self, column: str) -> pd.DataFrame:
    
    return self.dataset[column].mean()

In [84]:
class ExchangeProvider:
    currencies = {}

    def __init__(self, tables_wanted, currency_symbols):
        self.symbol_map = currency_symbols
        for table_name in tables_wanted:
            json_nbp_curr = re.get(f"http://api.nbp.pl/api/exchangerates/tables/{table_name}/last/1/?format=json").json()
            for rate in json_nbp_curr[0]['rates']:
                self.currencies[rate['code']] = rate['mid']

    def get_rate(self, from_currency, to_currency):
        if from_currency == to_currency:
            return 1

        from_rate = 0
        to_rate = 0

        if to_currency == "PLN":
            return  self.currencies[from_currency]
        elif from_currency == "PLN":
            return 1 / self.currencies[to_currency]
        else:
            try:
                from_rate = self.currencies[from_currency]
            except Exception as e:
                print(f"{e}")
                return -1
            
            try:
                to_rate = self.currencies[to_currency]
            except Exception as e:
                print(f"{e}")
                return -1
            
            return from_rate / to_rate
        
    def exchange_symbol_for_code(self, symbol: str) -> str:
        return self.symbol_map[symbol]


In [85]:
with open('./currencies.json') as f:
    currencies = json.load(f)

exchange_provider = ExchangeProvider(["a", "b"], currencies)

In [176]:
with open('./kaggle.json') as f: 
    credentials = json.load(f)

a = A(credentials['username'], credentials['key'], currencies)
column_type_map = {
    "role": str,
    "country": str,
    "company": str,
    "company_rating": float,
    "job_title": str,
    "median": str,
    "low": str,
    "high": str,
    "unit": str
}
a.load_dataset("imbishal7", "glassdoor-salary", "salaries.csv", 1, column_type_map)
a.process_data()
a.get_dataset()

/var/folders/wy/8f4lwcwd4pz4p8qrqmglgpjr0000gn/T/ipykernel_38693/1250365914.py:60: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  self.dataset['Currency'].fillna(currency_codes, inplace=True)
/var/folders/wy/8f4lwcwd4pz4p8qrqmglgpjr0000gn/T/ipykernel_38693/1250365914.py:65: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are sett

,role,country,company,company_rating,job_title,median,low,high,unit,Currency
0,Data Scientist,Japan,CTW,2.9,Data Scientist,1500000.0,1000000.0,18000000.0,/yr,JPY
1,Data Scientist,Japan,FUJIFILM,3.7,Data Scientist,5000000.0,5000000.0,7000000.0,/yr,JPY
3,Data Scientist,Japan,Data4Cʼs,3.0,Data Scientist,350000.0,300000.0,402000.0,/mo,JPY
4,Data Scientist,Japan,Deloitte,4.0,Data Scientist,6250000.0,5000000.0,8000000.0,/yr,JPY
5,Data Scientist,Japan,Indeed,4.1,Data Scientist,20500000.0,20000000.0,21000000.0,/yr,JPY
...,...,...,...,...,...,...,...,...,...,...
401185,Data Engineer,Romania,FinProm,2.9,Data Entry Operator,3716.0,4000.0,4000.0,/mo,RON
401186,Data Engineer,Romania,GRS,3.0,Data Entry Operator,1612.0,2000.0,2000.0,/mo,RON
401187,Data Engineer,Romania,Woods Valldata,2.6,Data Entry Operator,1648.0,2000.0,2000.0,/yr,RON
401188,Data Engineer,Romania,FM Logistic,3.5,Data Entry Operator,4908.0,5000.0,5000.0,/mo,RON


In [174]:
a.select_by_hour_rate()

,role,country,company,company_rating,job_title,median,low,high,unit,Currency
0,Data Scientist,Japan,CTW,2.9,Data Scientist,781.250000,520.833333,9375.000000,/hr,JPY
1,Data Scientist,Japan,FUJIFILM,3.7,Data Scientist,2604.166667,2604.166667,3645.833333,/hr,JPY
3,Data Scientist,Japan,Data4Cʼs,3.0,Data Scientist,2187.500000,1875.000000,2512.500000,/hr,JPY
4,Data Scientist,Japan,Deloitte,4.0,Data Scientist,3255.208333,2604.166667,4166.666667,/hr,JPY
5,Data Scientist,Japan,Indeed,4.1,Data Scientist,10677.083333,10416.666667,10937.500000,/hr,JPY
...,...,...,...,...,...,...,...,...,...,...
401185,Data Engineer,Romania,FinProm,2.9,Data Entry Operator,23.225000,25.000000,25.000000,/hr,RON
401186,Data Engineer,Romania,GRS,3.0,Data Entry Operator,10.075000,12.500000,12.500000,/hr,RON
401187,Data Engineer,Romania,Woods Valldata,2.6,Data Entry Operator,0.858333,1.041667,1.041667,/hr,RON
401188,Data Engineer,Romania,FM Logistic,3.5,Data Entry Operator,30.675000,31.250000,31.250000,/hr,RON


### Wnioski